<a href="https://colab.research.google.com/github/CarolineLaure/One_Qubit_TensorFlow_example/blob/master/three_level_system_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Now let have the Hamiltonian of the form H = - $\Delta_1$|e><e| - $\delta_{total}$|r><r| - $\Omega_1/2$(|g><e| + |e><g|)- $\Omega_2/2$(|e><r| + |r><e|) where |g>, |e>, |r> are respectively the ground, intermediate and the Rydberg state. The goal is to optimize the overlap between the ground and the Rydberg state

In [1]:
pip install qutip

     |████████████████████████████████| 3.5MB 2.8MB/s 
  Created wheel for qutip: filename=qutip-4.5.0-cp36-cp36m-linux_x86_64.whl size=12885007 sha256=a8c62b0a3043e02ca1b6bfcea247996a93d7324f09dad48a992dbc5c869fc002
  Stored in directory: /root/.cache/pip/wheels/c9/85/e8/3fbad9a0816141b4c5f1d73d8a880ed91265fea84192cbe37b
Successfully built qutip


In [16]:
from __future__ import absolute_import, division, print_function, unicode_literals
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import time
import qutip as qt
import matplotlib as plt 
print(tf.__version__) 

2.2.0-rc2


In [0]:
start = time.time()

# De leseleuc parameters

delta_1 = 2 * np.pi * 560 * 10**6
sigma_total = 2 * np.pi * 13 * 10**6
Omega_1 = 2 * np.pi * 60 * 10**6
Omega_2 = 2 * np.pi * 36 * 10**6

In [18]:
class Propagator:
    def __init__(self, no_of_steps, dim, delta_t, delta_1, sigma_total, Omega_1, Omega_2):

        # Define my Hamiltonian

        self.delta_1 = delta_1
        self.sigma_total = sigma_total
        self.Omega_1 = Omega_1
        self.Omega_2 = Omega_2
        self.delta_t = delta_t
        
        g, e, r = qt.qutrit_basis()

        Hamiltonian = - self.delta_1 * e * e.dag() - self.sigma_total * r * r.dag() - self.Omega_1/2 * (g * e.dag() + e * g.dag()) 
        - self.Omega_2/2 * (e * r.dag() + r * e.dag())


        self.delta_t=delta_t
        self.dim=3
        self.initial_state= tf.constant(g, dtype=tf.complex128)
        self.final_state= tf.constant(r, dtype=tf.complex128)
        

        self.ctrl_amplitude = tf.Variable(tf.zeros([no_of_steps, 4], dtype=tf.float64), dtype=tf.float64)  # control amplitude $\Omega$

        self.generators =  tf.stack([
                                     tf.constant(- delta_1 * e * e.dag(), dtype=tf.complex128),
                                     tf.constant(- sigma_total * r * r.dag(), dtype=tf.complex128),
                                     tf.constant(- Omega_1 * 0.5 * (g * e.dag() + e * g.dag()), dtype=tf.complex128),
                                     tf.constant(- Omega_2 * 0.5 * (e * r.dag() + r * e.dag()), dtype=tf.complex128),
                          
                                     ])

        self.contraction_array=[]
        contraction_array_length = int(np.floor(np.log2(no_of_steps)))
        temp_no_of_steps= no_of_steps

        for i in range(contraction_array_length):
          self.contraction_array.append(bool(np.mod(temp_no_of_steps, 2))) 
          temp_no_of_steps = np.floor(temp_no_of_steps/2)

    def exponential(self):
          regularize_amplitudes = 1/np.sqrt(2)*tf.math.tanh(self.ctrl_amplitude)
          exponent = -1j*self.delta_t*(tf.linalg.tensordot(tf.cast(regularize_amplitudes, dtype=tf.complex128), self.generators, 1))
          return tf.linalg.expm(exponent)

    def propagate(self):
          step_exps=self.exponential()
          for is_odd in self.contraction_array:
            if is_odd:
              odd_exp=step_exps[-1, :, :]
              step_exps = tf.linalg.matmul(step_exps[1::2, :, :], step_exps[0:-1:2, :, :])
              step_exps = tf.concat([step_exps[0:-1, :, :], [tf.linalg.matmul(odd_exp, step_exps[-1, :, :])]], 0)
            else:
              step_exps = tf.linalg.matmul(step_exps[1::2, :, :], step_exps[0::2, :, :])
          return tf.squeeze(step_exps)

    @tf.function
    def infidelity(self):
          propagator=self.propagate()
          intermediate_state = tf.linalg.matvec(propagator, tf.transpose(self.initial_state))
          overlap = tf.tensordot(tf.math.conj(intermediate_state), self.final_state, axes=1)
          return 1-tf.math.real(tf.math.conj(overlap)*overlap)

propagator = Propagator(2000, 3, 10**-9, delta_1, sigma_total, Omega_1, Omega_2)

optimizer = tf.keras.optimizers.Adam(0.01)

propagator.ctrl_amplitude.assign(
    tf.random.uniform([2000, 4], -1, 1, dtype=tf.float64)
)

#propagator.infidelity()

@tf.function  
def optimization_step():
  with tf.GradientTape() as tape:
    infidelity = propagator.infidelity()
  gradients = tape.gradient(infidelity, [propagator.ctrl_amplitude])
  optimizer.apply_gradients(zip(gradients, [propagator.ctrl_amplitude]))
  return infidelity


steps = range(100)
for step in steps:
  current_infidelity = optimization_step()
  print('step %2d: infidelity=%2.5f' % (step, current_infidelity))



#final_pulse = propagator.ctrl_amplitude  
        
end = time.time()
print('Run Time: %2.4f seconds' %(end-start))

step  0: infidelity=0.97051
step  1: infidelity=0.52874
step  2: infidelity=0.71680
step  3: infidelity=0.95033
step  4: infidelity=0.62262
step  5: infidelity=0.86765
step  6: infidelity=0.66892
step  7: infidelity=0.83974
step  8: infidelity=0.57054
step  9: infidelity=0.84858
step 10: infidelity=0.65567
step 11: infidelity=0.49411
step 12: infidelity=0.27467
step 13: infidelity=0.05697
step 14: infidelity=0.34197
step 15: infidelity=0.03488
step 16: infidelity=0.18007
step 17: infidelity=0.02120
step 18: infidelity=0.09219
step 19: infidelity=0.12874
step 20: infidelity=0.06215
step 21: infidelity=0.18838
step 22: infidelity=0.15427
step 23: infidelity=0.08261
step 24: infidelity=0.05017
step 25: infidelity=0.03504
step 26: infidelity=0.06988
step 27: infidelity=0.06895
step 28: infidelity=0.03818
step 29: infidelity=0.04037
step 30: infidelity=0.03850
step 31: infidelity=0.03292
step 32: infidelity=0.06038
step 33: infidelity=0.01735
step 34: infidelity=0.02235
step 35: infidelity=